In [ ]:
import pandas as pd
import numpy as np
aggregated_df = pd.read_parquet("/workspace/recomAI/aggregated_df_250625_info.parquet")

# 필요한 컬럼만 선택
fin_aggregated_df = aggregated_df[['GNDR_CD', 'INS_AGE', 'JOB_GRD_CD', 'INJR_GRD', 'DRV_USG_DIV_CD', 'CHN_DIV',
                                   'SBCP_YYMM', 'UNT_PD_NM', 'SLZ_PREM', 'PY_INS_PRD_NAME', 'LWRT_TMN_RFD_TP_CD',
                                   'PY_EXEM_TP_CD', 'HNDY_ISP_TP_NM', 'PLAN_NM', 'PD_COV_NM', 'SBC_AMT',
                                   'cov_치매', 'cov_심장질환', 'cov_후유장해', 'cov_뇌혈관질환', 'cov_암', 'cov_사망',
                                   'cov_기타', 'cov_입원비(일당)', 'cov_운전자', 'cov_치아,화상,골절', 'cov_수술비',
                                   'cov_의료비', 'cov_법률,배상책임']].copy()

# 보험료 구간 설정
bins = [-np.inf, 50000, 100000, 150000, 200000, np.inf]
labels = ['5만원 이하', '5~10만원 이하', '10~15만원 이하', '15~20만원 이하', '20만원 초과']
fin_aggregated_df['tar_prem'] = pd.cut(fin_aggregated_df['SLZ_PREM'], bins=bins, labels=labels)

# 테마 컬럼 리스트
theme_columns = ['cov_치매', 'cov_심장질환', 'cov_후유장해', 'cov_뇌혈관질환', 'cov_암', 'cov_사망',
                 'cov_입원비(일당)', 'cov_운전자', 'cov_치아,화상,골절',
                 'cov_수술비', 'cov_의료비', 'cov_법률,배상책임']

# tar_theme 만들기: 값이 1보다 큰 테마명을 리스트로 저장
def extract_themes(row):
    themes = [col.replace('cov_', '') for col in theme_columns if row[col] >= 1]
    return ', '.join(themes) if themes else '없음'

fin_aggregated_df['tar_theme'] = fin_aggregated_df.apply(extract_themes, axis=1)
fin_aggregated_df = fin_aggregated_df.drop(columns=theme_columns)

In [ ]:
# 유저정보 - 성별, 나이, 직업급수, 상해등급, 운전용도, 채널, 희망보험료, 희망테마
# 'GNDR_CD', 'INS_AGE', 'JOB_GRD_CD', 'INJR_GRD', 'DRV_USG_DIV_CD' 'CHN_DIV'

# 아이템정보 - 청약년월, 상품이름, 보험료, 납만기, 저율해지, 납입면제, 간편심사유형, 플랜, 무해지유형, 납입면제유형, 담보명, 가입금액
# 'SBCP_YYMM', 'UNT_PD_NM', 'SLZ_PREM', 'PY_INS_PRD_NAME', 'LWRT_TMN_RFD_TP_CD', 'PY_EXEM_TP_CD', 'HNDY_ISP_TP_NM', 'PLAN_NM', 'PD_COV_NM', 'SBC_AMT', 

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_path = './MiniLM-L12-v2/0_Transformer'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

sentences = ["테스트 문장입니다."]
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"].unsqueeze(-1)
    pooled = (embeddings * attention_mask).sum(1) / attention_mask.sum(1)

print("✅ 수동 임베딩:", pooled.shape)

In [ ]:
import pandas as pd
import re

class InsuranceDataConverter:
    def __init__(self):
        # 코드 매핑 사전들
        self.gender_map = {
            '1': '남성',
            '2': '여성'
        }
        
        self.job_grade_map = {
            '1': '1급 사무직',
            '2': '2급 일반직',
            '3': '3급 위험직',
        }
        
        self.injury_grade_map = {
            '1': '1급 매우낮음',
            '2': '2급 낮음', 
            '3': '3급 낮음',
            '4': '4급 보통',
            '5': '5급 높음',
            '6': '6급 높음',
            '7': '7급 매우높음',
            '8': '8급 매우높음',
            '9': '9급 고위험',
            '10': '10급 고위험'

        }
        
        self.payment_exemption_map = {
            '00': '납입면제 미적용형',
            '01': '납입면제1형',
            '02': '납입면제2형',
            '03': '납입면제3형',
            '04': '납입면제4형',
            '05': '납입면제5형',
            '06': '납입면제6형'
        }
        
        self.surrender_refund_map = {
            '00': '표준형',
            '01': '해지환급금미지급형',
            '02': '해지환급금미지급형',
            '03': '해지환급금미지급형',
            '04': '해지환급금50%지급형',
            '05': '해지환급금미지급형',
            '06': '해지환급금미지급형',
            '07': '해지환급금지급형'
        }
        
        self.health_declaration_map = {
            '01': '일반고지형',
            '02': '건강고지형 6년',
            '03': '건강고지형 7년',
            '04': '건강고지형 8년',
            '05': '건강고지형 9년',
            '06': '건강고지형 10년'
        }

    def decode_value(self, value, mapping_dict):
        """코드값을 의미있는 텍스트로 변환"""
        if pd.isna(value):
            return "정보없음"
        return mapping_dict.get(str(value), str(value))

    def format_premium(self, premium):
        """보험료를 읽기 쉬운 형태로 포맷"""
        if pd.isna(premium):
            return "보험료 정보없음"
        
        premium = int(premium)
        return self.format_amount_korean(premium)
    
    def format_amount_korean(self, amount):
        """금액을 한국어 단위로 포맷 (예: 50000 -> 5만원)"""
        if amount == 0:
            return "0원"
        
        # 억 단위
        if amount >= 100000000:
            uk = amount // 100000000
            remainder = amount % 100000000
            if remainder == 0:
                return f"{uk}억원"
            elif remainder >= 10000000:  # 천만 단위
                man = remainder // 10000000
                return f"{uk}억 {man}천만원"
            elif remainder >= 10000:  # 만 단위
                man = remainder // 10000
                return f"{uk}억 {man}만원"
            else:
                return f"{uk}억 {remainder}원"
        
        # 만 단위 
        elif amount >= 10000:
            man = amount // 10000
            remainder = amount % 10000
            if remainder == 0:
                return f"{man}만원"
            else:
                return f"{man}만 {remainder}원"
        
        # 만 미만
        else:
            return f"{amount}원"

    import re

    def preprocess_product_name(self, name: str) -> str:
        """
        보험 상품명 전처리 함수

        Args:
            name: 원본 상품명

        Returns:
            전처리된 상품명
        """
        # 1. '(무)' 제거
        name = re.sub(r'\(무\)', '', name)

        # 2. '메리츠' 제거
        name = re.sub(r'\b메리츠\b', '', name)

        # 3. '2504' 등 숫자 연도 제거 (2504, 2505 등)
        name = re.sub(r'\d{4}', '', name)

        # 4. 괄호 내 텍스트 제거 (단, 갱신형/세만기형은 남김)
        # 예: (통합간편심사형) → 제거
        name = re.sub(r'\((?!.*갱신형|세만기형).*?\)', '', name)

        # 5. 괄호 자체 제거 (남은 경우)
        name = re.sub(r'[()]', '', name)

        # 6. 공백 정리
        name = re.sub(r'\s+', ' ', name).strip()

        return name


    def format_coverage_and_amounts(self, coverage_str, amount_str):
        """담보명과 가입금액을 매칭하여 포맷"""
        if pd.isna(coverage_str) or pd.isna(amount_str):
            return "담보 정보없음"
        
        # 담보명 파싱 (! 구분)
        coverages = coverage_str.split('!')
        coverages = [cov.strip() for cov in coverages if cov.strip()]
        
        # 가입금액 파싱 (, 구분)
        amounts = str(amount_str).split(',')
        amounts = [amt.strip() for amt in amounts if amt.strip()]
        
        # 담보와 금액 매칭
        coverage_list = []
        for i, coverage in enumerate(coverages):
            if i < len(amounts):
                amount = amounts[i]
                # 금액 포맷팅 (한국어 단위로)
                try:
                    amount_int = int(amount)
                    formatted_amount = self.format_amount_korean(amount_int)
                except:
                    formatted_amount = amount
                
                # 담보명 정리 (불필요한 기호 제거)
                clean_coverage = re.sub(r'[갱신형|!\[\]()]', '', coverage).strip()
                coverage_list.append(f"{clean_coverage} {formatted_amount}")
            else:
                clean_coverage = re.sub(r'[갱신형|!\[\]()]', '', coverage).strip()
                coverage_list.append(clean_coverage)
        
        return ", ".join(coverage_list[:3]) + ("..." if len(coverage_list) > 3 else "")

    def format_target_theme(self, theme_str):
        """타겟 테마를 읽기 쉽게 포맷"""
        if pd.isna(theme_str):
            return "특별한 관심사항 없음"
        
        themes = str(theme_str).split(',')
        themes = [theme.strip() for theme in themes if theme.strip()]
        
        theme_map = {
            '사망': '사망',
            '암': '암',
            '치매': '치매', 
            '뇌질환': '뇌혈관',
            '심장질환': '심장질환',
            '수술비': '수술비',
            '간병': '간병',
            '치아': '치아',
            '화상': '화상',
            '골절': '골절'
        }
        
        formatted_themes = []
        for theme in themes[:4]:  # 최대 4개만
            formatted_themes.append(theme_map.get(theme, theme))
        
        return ", ".join(formatted_themes)

    def convert_to_query_value_pair(self, row):
        """단일 행을 Query-Value pair로 변환"""
        
        # === QUERY: 사용자 정보 (고객 프로필) ===
        gender = self.decode_value(row['GNDR_CD'], self.gender_map)
        age = f"{row['INS_AGE']}세" if not pd.isna(row['INS_AGE']) else "연령 정보없음"
        job_grade = self.decode_value(row['JOB_GRD_CD'], self.job_grade_map)
        injury_grade = self.decode_value(row['INJR_GRD'], self.injury_grade_map)
        # driving_usage = self.decode_value(row['DRV_USG_DIV_CD'], self.driving_usage_map)
        # channel = self.decode_value(row['CHN_DIV'], self.channel_map)
        target_premium = row.get('tar_prem', '희망보험료 정보없음')
        target_theme = self.format_target_theme(row.get('tar_theme', ''))
        
        query = (
            f"{age} {gender} 고객으로 직업등급 {job_grade}, 상해등급 {injury_grade}에 해당합니다. "
            f"희망하는 보험료는 {target_premium}이고 {target_theme} 테마에 특별한 관심이 있습니다."
        )
        
        # === VALUE: 보험상품 정보 (증권 정보) ===
        product_name = row.get('UNT_PD_NM', '상품명 정보없음')
        product_name = self.preprocess_product_name(product_name)
        premium = self.format_premium(row.get('SLZ_PREM'))
        payment_period = row.get('PY_INS_PRD_NAME', '납입기간 정보없음')
        surrender_type = self.decode_value(row.get('LWRT_TMN_RFD_TP_CD'), self.surrender_refund_map)
        payment_exemption = self.decode_value(row.get('PY_EXEM_TP_CD'), self.payment_exemption_map)
        simple_review = row.get('HNDY_ISP_TP_NM', '심사유형 정보없음')
        plan_name = row.get('PLAN_NM', '')
        
        # 담보 정보
        coverage_info = self.format_coverage_and_amounts(
            row.get('PD_COV_NM'), 
            row.get('SBC_AMT')
        )
        
        value = (
            f"{product_name} 상품으로 월 보험료 {premium}입니다. "
            f"납입조건은 {payment_period}이며 {surrender_type} 방식을 적용합니다. "
            f"{payment_exemption} 조건이 포함되고 {simple_review}으로 간편하게 가입 가능합니다."
        )
        
        if plan_name and str(plan_name) != 'None' and str(plan_name).strip():
            value += f" {plan_name} 플랜이 적용됩니다."
        
        value += f" 주요 보장내용: {coverage_info}"
        
        return {
            'date' : row.get('SBCP_YYMM'),
            'query': query.strip(),
            'value': value.strip(),
            'label': 1
        }

    def convert_dataframe(self, df):
        """전체 데이터프레임을 Query-Value pair로 변환"""
        results = []
        
        for idx, row in df.iterrows():
            converted = self.convert_to_query_value_pair(row)
            converted['original_index'] = idx
            results.append(converted)
        
        return pd.DataFrame(results)


# 사용 예시
if __name__ == "__main__":
    df = fin_aggregated_df #.head(1000)
    
    # 변환기 초기화 및 실행
    converter = InsuranceDataConverter()
    result_df = converter.convert_dataframe(df)

    # Sentence Transformer용 데이터 준비
    print("🤖 Sentence Transformer 학습용 데이터:")
    print("Queries:", result_df['query'].tolist()[:3])
    print("Values:", result_df['value'].tolist()[:3])

In [ ]:
def show_tokenization(trainer: InsuranceEmbeddingTrainer, df: pd.DataFrame, column: str = 'query', max_rows: int = 3):
    """
    특정 컬럼의 텍스트를 tokenizer로 토큰화하여 확인하는 함수
    
    Args:
        trainer: InsuranceEmbeddingTrainer 인스턴스 (tokenizer 포함)
        df: result_df 형태의 데이터프레임
        column: 토큰화할 컬럼명 ('query' 또는 'value')
        max_rows: 출력할 샘플 수
    """
    if trainer.tokenizer is None:
        raise ValueError("tokenizer가 초기화되지 않았습니다. load_pretrained_model()을 먼저 호출하세요.")
    
    sample_texts = df[column].head(max_rows).tolist()
    
    for i, text in enumerate(sample_texts):
        tokens = trainer.tokenizer.tokenize(text)
        token_ids = trainer.tokenizer.convert_tokens_to_ids(tokens)
        print(f"\n--- 샘플 {i+1} ---")
        print(f"원문: {text}")
        print(f"토큰: {tokens}")
        print(f"토큰 ID: {token_ids}")

In [ ]:
# 모델 로딩
trainer = InsuranceEmbeddingTrainer(
    model_path='./MiniLM-L12-v2/0_Transformer'
)
trainer.load_pretrained_model()

# result_df의 query 컬럼 토큰화 보기
show_tokenization(trainer, result_df, column='query')

# value 컬럼도 보고 싶다면
show_tokenization(trainer, result_df, column='value')

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import logging
import random
from typing import List, Tuple, Dict
import os
from datetime import datetime

# 로깅 설정
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

class InsuranceEmbeddingTrainer:
    def __init__(self, 
                 model_path: str = './MiniLM-L12-v2/0_Transformer',
                 output_path: str = './trained_insurance_model_retoken'):
        """
        보험 추천을 위한 Embedding 학습기
        
        Args:
            model_path: 사전 학습된 모델 경로
            output_path: 학습된 모델 저장 경로
        """
        self.model_path = model_path
        self.output_path = output_path
        self.tokenizer = None
        self.model = None
        self.sentence_transformer = None
        
    def load_pretrained_model(self):
        """사전 학습된 모델 로드"""
        logging.info(f"Loading pretrained model from: {self.model_path}")
        
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            self.model = AutoModel.from_pretrained(self.model_path)
            logging.info("✅ Pretrained model loaded successfully")
        except Exception as e:
            logging.error(f"❌ Error loading pretrained model: {e}")
            # Fallback to online model
            logging.info("Falling back to online model...")
            self.sentence_transformer = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    def get_embeddings_manual(self, sentences: List[str]) -> torch.Tensor:
        """수동으로 임베딩 추출 (사전 학습된 모델 사용)"""
        if self.tokenizer is None or self.model is None:
            raise ValueError("Pretrained model not loaded. Call load_pretrained_model() first.")
        
        inputs = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state
            attention_mask = inputs["attention_mask"].unsqueeze(-1)
            # Mean pooling
            pooled = (embeddings * attention_mask).sum(1) / attention_mask.sum(1)
        
        return pooled
    
    def prepare_sentence_transformer(self):
        """Sentence Transformer 모델 준비"""
        if self.sentence_transformer is None:
            # 로컬 모델을 Sentence Transformer로 변환
            logging.info("Converting pretrained model to Sentence Transformer...")
            self.sentence_transformer = SentenceTransformer(self.model_path).to('cuda')
          
    
    def prepare_training_data(self, df: pd.DataFrame) -> List[InputExample]:
        """
        올리브영 방식: Positive pairs만 사용하여 학습 데이터 준비
        
        Args:
            df: 학습 데이터프레임 (query, value, label 컬럼 필요)
            
        Returns:
            InputExample 리스트 (positive pairs만)
        """
        examples = []
        
        logging.info(f"Preparing training data with {len(df)} samples")
        
        # Label=1인 positive pairs만 사용
        positive_df = df[df['label'] == 1]
        
        for _, row in positive_df.iterrows():
            # Label 없이 positive pair만 생성
            # MultipleNegativesRankingLoss가 배치 내에서 자동으로 negative sampling 수행
            examples.append(InputExample(texts=[row['query'], row['value']]))
        
        logging.info(f"Created {len(examples)} positive pairs")
        logging.info("Negative pairs will be automatically generated in-batch by MultipleNegativesRankingLoss")
        
        # 데이터 셔플
        random.shuffle(examples)
        return examples
    
  
    def create_simple_evaluator(self, eval_df: pd.DataFrame):
        """
        간단한 평가자 생성 (상관계수 대신 정확도 기반)
        """
        from sentence_transformers.evaluation import BinaryClassificationEvaluator
        
        # 평가 데이터를 binary classification 형태로 변환
        sentences1 = []
        sentences2 = []
        scores = []
        
        queries = eval_df['query'].tolist()
        values = eval_df['value'].tolist()
        
        # Positive pairs (실제 매칭)
        for query, value in zip(queries, values):
            sentences1.append(query)
            sentences2.append(value)
            scores.append(1)  # positive
        
        # Negative pairs (랜덤 매칭)
        for i in range(min(len(queries), 20)):  # 최대 20개 negative
            neg_idx = (i + len(queries) // 2) % len(values)  # 다른 인덱스 선택
            sentences1.append(queries[i])
            sentences2.append(values[neg_idx])
            scores.append(0)  # negative
        
        logging.info(f"Created binary evaluator with {len(sentences1)} pairs")
        logging.info(f"Positive: {scores.count(1)}, Negative: {scores.count(0)}")
        
        return BinaryClassificationEvaluator(
            sentences1=sentences1,
            sentences2=sentences2,
            labels=scores,
            name="insurance_binary_eval"
        )
    
    def train_model_with_safe_evaluator(self, 
                                       train_df: pd.DataFrame,
                                       eval_df: pd.DataFrame = None,
                                       epochs: int = 4,
                                       batch_size: int = 16,
                                       learning_rate: float = 2e-5,
                                       warmup_steps: int = None):
        """
        안전한 평가자를 사용한 학습
        """
        # Sentence Transformer 준비
        self.prepare_sentence_transformer()
        
        # 학습 데이터 준비 (positive pairs만)
        train_examples = self.prepare_training_data(train_df)
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
        
        # Loss function 설정 (MultipleNegativesRankingLoss)
        train_loss = losses.MultipleNegativesRankingLoss(model=self.sentence_transformer)
        
        # Warmup steps 계산
        if warmup_steps is None:
            warmup_steps = int(len(train_dataloader) * epochs * 0.1)
        
        # 안전한 평가자 생성
        evaluator = None
        if eval_df is not None and len(eval_df) >= 5:
            try:
                evaluator = self.create_simple_evaluator(eval_df)
                logging.info("✅ Safe binary evaluator created")
            except Exception as e:
                logging.warning(f"⚠️ Failed to create safe evaluator: {e}")
                evaluator = None
        
        # 모델 학습
        logging.info("🚀 Starting training with safe evaluator...")
        logging.info(f"📊 Training examples: {len(train_examples)}")
        logging.info(f"📦 Batch size: {batch_size}")
        logging.info(f"🔄 Epochs: {epochs}")
        logging.info(f"🔥 Learning rate: {learning_rate}")
        logging.info(f"⚡ Warmup steps: {warmup_steps}")
        logging.info(f"📈 Safe evaluator enabled: {evaluator is not None}")
        
        if evaluator is not None:
            self.sentence_transformer.fit(
                train_objectives=[(train_dataloader, train_loss)],
                epochs=epochs,
                warmup_steps=warmup_steps,
                evaluator=evaluator,
                evaluation_steps=35,  # 더 자주 평가 (매 에포크마다)
                output_path=self.output_path,
                save_best_model=True,
                show_progress_bar=True,
                optimizer_params={'lr': learning_rate},
                use_amp=False,  # Mixed precision 비활성화
                checkpoint_save_steps=None,  # 체크포인트 저장 비활성화
                checkpoint_save_total_limit=None
            )
        else:
            # Fallback: 평가자 없이 학습
            self.sentence_transformer.fit(
                train_objectives=[(train_dataloader, train_loss)],
                epochs=epochs,
                warmup_steps=warmup_steps,
                output_path=self.output_path,
                show_progress_bar=True,
                optimizer_params={'lr': learning_rate}
            )
        
        logging.info(f"✅ Training completed. Saved to: {self.output_path}")
    
    def load_trained_model(self, model_path: str = None):
        """학습된 모델 로드"""
        if model_path is None:
            model_path = self.output_path
        
        logging.info(f"Loading trained model from: {model_path}")
        self.sentence_transformer = SentenceTransformer(model_path)
    
    def evaluate_model_performance(self, test_df: pd.DataFrame) -> Dict:
        """모델 성능 평가"""
        if self.sentence_transformer is None:
            raise ValueError("Model not loaded. Please train or load a model first.")
        
        # 임베딩 생성
        queries = test_df['query'].tolist()
        values = test_df['value'].tolist()
        labels = test_df['label'].tolist()
        
        logging.info("🔍 Generating embeddings for evaluation...")
        query_embeddings = self.sentence_transformer.encode(queries, show_progress_bar=True, device='cuda')
        value_embeddings = self.sentence_transformer.encode(values, show_progress_bar=True, device='cuda')
        
        # 유사도 계산
        similarities = []
        for q_emb, v_emb in zip(query_embeddings, value_embeddings):
            sim = cosine_similarity([q_emb], [v_emb])[0][0]
            similarities.append(sim)
        
        # 성능 지표 계산
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
        
        # 유사도를 이진 분류로 변환 (임계값 0.5)
        predictions = [1 if sim > 0.5 else 0 for sim in similarities]
        
        metrics = {
            'accuracy': accuracy_score(labels, predictions),
            'precision': precision_score(labels, predictions, zero_division=0),
            'recall': recall_score(labels, predictions, zero_division=0),
            'f1_score': f1_score(labels, predictions, zero_division=0),
            'avg_similarity': np.mean(similarities),
            'std_similarity': np.std(similarities)
        }
        
        return metrics
    
    def recommend_products(self, 
                          user_query: str, 
                          product_values: List[str], 
                          top_k: int = 5) -> List[Tuple[int, float]]:
        """사용자 쿼리에 대해 상품 추천"""
        if self.sentence_transformer is None:
            raise ValueError("Model not loaded. Please train or load a model first.")
        
        # 임베딩 생성
        query_embedding = self.sentence_transformer.encode([user_query])
        product_embeddings = self.sentence_transformer.encode(product_values)
        
        # 코사인 유사도 계산
        similarities = cosine_similarity(query_embedding, product_embeddings)[0]
        
        # 상위 k개 추천
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        recommendations = [(idx, similarities[idx]) for idx in top_indices]
        
        return recommendations


def main_training_pipeline(result_df: pd.DataFrame):
    """
    메인 학습 파이프라인
    
    Args:
        result_df: InsuranceDataConverter로 변환된 데이터프레임
    """
    
    logging.info("🎯 보험 추천 모델 학습 파이프라인 시작")
    
    # 1. 데이터 전처리 및 분할
    logging.info("📊 데이터 전처리 및 분할 중...")
    
    # 날짜별로 분할 (시간순 분할)
    if 'date' in result_df.columns:
        unique_dates = sorted(result_df['date'].unique())
        train_dates = unique_dates[:-1]  # 마지막 날짜 제외하고 학습
        test_dates = [unique_dates[-1]]  # 마지막 날짜를 테스트
        
        train_df = result_df[result_df['date'].isin(train_dates)]
        test_df = result_df[result_df['date'].isin(test_dates)]
        
        logging.info(f"📅 Train dates: {train_dates}")
        logging.info(f"📅 Test dates: {test_dates}")
    else:
        # 날짜 정보가 없으면 랜덤 분할
        train_df, test_df = train_test_split(result_df, test_size=0.2, random_state=42)
    
    # 학습/검증 분할
    train_df, eval_df = train_test_split(train_df, test_size=0.1, random_state=42)
    
    logging.info(f"📈 Train samples: {len(train_df)}")
    logging.info(f"📊 Eval samples: {len(eval_df)}")
    logging.info(f"🧪 Test samples: {len(test_df)}")
    
    # 2. 트레이너 초기화
    trainer = InsuranceEmbeddingTrainer(
        model_path='./MiniLM-L12-v2/0_Transformer',
        output_path='./trained_insurance_model_retoken'
    )
    
    # 3. 사전 학습된 모델 로드 (선택적)
    try:
        trainer.load_pretrained_model()
        
        # 사전 학습된 모델로 샘플 임베딩 테스트
        sample_sentences = train_df['query'].head(3).tolist()
        embeddings = trainer.get_embeddings_manual(sample_sentences)
        logging.info(f"✅ 사전 학습된 모델 임베딩 테스트: {embeddings.shape}")
        
    except Exception as e:
        logging.warning(f"⚠️ 사전 학습된 모델 로드 실패: {e}")
        logging.info("🌐 온라인 모델 사용으로 전환")
    
    # 4. 안전한 평가자와 함께 학습
    logging.info("🚀 안전한 평가자와 함께 학습 시작...")
    trainer.train_model_with_safe_evaluator(
        train_df=train_df,
        eval_df=eval_df,
        epochs=4,
        batch_size=16,
        learning_rate=2e-5
    )
    
    # 5. 모델 성능 평가
    logging.info("📊 모델 성능 평가 중...")
    metrics = trainer.evaluate_model_performance(test_df)
    
    print("\n" + "="*50)
    print("📊 모델 성능 평가 결과")
    print("="*50)
    for metric, value in metrics.items():
        print(f"{metric:15}: {value:.4f}")
    print("="*50)
    
    # 6. 추천 시스템 테스트
    logging.info("🎯 추천 시스템 테스트 중...")
    
    # 테스트 쿼리 생성
    test_query = test_df['query'].iloc[0]
    test_values = test_df['value'].head(10).tolist()
    
    recommendations = trainer.recommend_products(
        user_query=test_query,
        product_values=test_values,
        top_k=5
    )
    
    print("\n" + "="*50)
    print("🎯 추천 시스템 테스트 결과")
    print("="*50)
    print(f"👤 사용자 쿼리: {test_query[:100]}...")
    print("\n📋 추천 상품:")
    for i, (idx, score) in enumerate(recommendations):
        print(f"{i+1}. 유사도: {score:.4f}")
        print(f"   상품: {test_values[idx][:100]}...")
        print()
    
    # 7. 모델 저장 확인
    if os.path.exists(trainer.output_path):
        logging.info(f"✅ 모델이 성공적으로 저장되었습니다: {trainer.output_path}")
    else:
        logging.error(f"❌ 모델 저장 실패: {trainer.output_path}")
    
    return trainer, metrics


# 실행 예시
if __name__ == "__main__":

    trainer, metrics = main_training_pipeline(result_df)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

class AttentionAnalyzer:
    """보험 추천 모델의 어텐션 패턴 분석 도구"""
    
    def __init__(self, model_path: str):
        self.model_path = model_path
        self.tokenizer = None
        self.model = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def load_model(self):
        """모델과 토크나이저 로드"""

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path, output_attentions=True)
        self.model.to(self.device)
        self.model.eval()
        print(f"✅ 모델 로드 완료: {self.model_path}")
       
    
    def clean_token(self, token: str) -> str:
        """토큰 정리 함수"""
        cleaned = token.replace('▁', '').replace('##', '')
        if cleaned in ['[CLS]', '[SEP]', '[PAD]', '<s>', '</s>', '<pad>', '<unk>']:
            return cleaned
        if not cleaned.strip():
            return '[SPACE]'
        return cleaned
    
    def get_attention_weights(self, text: str) -> Tuple[List[str], torch.Tensor]:
        """텍스트에 대한 어텐션 가중치 추출"""
        inputs = self.tokenizer(
            text, return_tensors="pt", padding=True, 
            truncation=True, max_length=128
        ).to(self.device)
        
        raw_tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        print(f"raw_tokens: {raw_tokens}")
        tokens = [self.clean_token(token) for token in raw_tokens]
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            attention_weights = torch.stack(outputs.attentions)
            attention_weights = attention_weights.squeeze(1)
        
        return tokens, attention_weights.cpu()
    
    def visualize_token_importance(self, text: str, layer_idx: int = -1, top_k: int = 10):
        """토큰 중요도 막대 그래프"""
        tokens, attention_weights = self.get_attention_weights(text)
        
        if layer_idx == -1:
            layer_idx = attention_weights.shape[0] - 1
        attention = attention_weights[layer_idx].mean(dim=0)  # 모든 헤드 평균
        
        cls_attention = attention[0, :].numpy()  # CLS 토큰의 어텐션
        
        df = pd.DataFrame({
            'token': tokens,
            'importance': cls_attention,
            'position': range(len(tokens))
        })

        df = df.sort_values('importance', ascending=False)
        
        # 특수 토큰 제외
        special_tokens = ['[CLS]', '[SEP]', '[PAD]', '<s>', '</s>', '<pad>', '[SPACE]']
        df_filtered = df[~df['token'].isin(special_tokens)].head(top_k)
        
        # 시각화
        plt.figure(figsize=(12, 6))
        bars = plt.bar(range(len(df_filtered)), df_filtered['importance'], 
                       color=plt.cm.viridis(df_filtered['importance'] / df_filtered['importance'].max()))
        
        plt.xlabel('Tokens', fontsize=12)
        plt.ylabel('Attention Weight', fontsize=12)
        plt.title(f'Token Importance Analysis (Layer {layer_idx})\nText: "{text[:50]}..."', 
                 fontsize=14, pad=20)
        plt.xticks(range(len(df_filtered)), df_filtered['token'], rotation=45, ha='right')
        
        # 값 표시
        for i, (bar, val) in enumerate(zip(bars, df_filtered['importance'])):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                    f'{val:.3f}', ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        return df_filtered
    
    def visualize_attention_heatmap(self, text: str, layer_idx: int = -1, head_idx: int = 0):
        """어텐션 히트맵 시각화"""
        tokens, attention_weights = self.get_attention_weights(text)
        
        if layer_idx == -1:
            layer_idx = attention_weights.shape[0] - 1
        
        attention = attention_weights[layer_idx, head_idx].numpy()
        
        # 토큰 길이 제한
        max_tokens = 100
        if len(tokens) > max_tokens:
            tokens = tokens[:max_tokens]
            attention = attention[:max_tokens, :max_tokens]
        
        # 토큰 길이 제한 (표시용)
        display_tokens = [token[:6] + '..' if len(token) > 8 else token for token in tokens]
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(
            attention, xticklabels=display_tokens, yticklabels=display_tokens,
            cmap='Blues', annot=False, cbar_kws={'label': 'Attention Weight'}
        )
        
        plt.title(f'Attention Heatmap\nLayer {layer_idx}, Head {head_idx}\nText: "{text[:50]}..."', 
                 fontsize=14, pad=20)
        plt.xlabel('Key Tokens', fontsize=12)
        plt.ylabel('Query Tokens', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

def analyze_test_samples(test_df: pd.DataFrame, 
                        model_path: str = "./trained_insurance_model_retoken",
                        sample_size: int = 3):
    """
    test_df에서 샘플을 뽑아서 어텐션 시각화
    
    Args:
        test_df: 테스트 데이터프레임 (query, value 컬럼 필요)
        model_path: 모델 경로
        sample_size: 분석할 샘플 개수
    """
    print("🔍 테스트 데이터 어텐션 분석 시작...")
    print(f"📊 test_df 크기: {test_df.shape}")
    print(f"📋 컬럼명: {list(test_df.columns)}")
    
    # 분석기 생성 및 모델 로드
    analyzer = AttentionAnalyzer(model_path)
    analyzer.load_model()
    
    # 샘플 데이터 추출
    sample_data = test_df.sample(n=min(sample_size, len(test_df)), random_state=42)
    
    print(f"\n📊 {len(sample_data)}개 샘플 분석 시작")
    print("=" * 60)
    
    results = []
    
    for i, (idx, row) in enumerate(sample_data.iterrows()):
        query = str(row['query'])
        value = str(row['value'])
        
        print(f"\n🔍 샘플 {i+1}/{len(sample_data)}")
        print(f"📝 쿼리: {query}...")
        print(f"🏷️ 상품: {value}...")
        print("-" * 40)
        
        # 1. 쿼리 토큰 중요도 분석
        print("📊 쿼리 토큰 중요도:")
        query_df = analyzer.visualize_token_importance(query, top_k=8)
        query_top_tokens = query_df['token'].head(3).tolist()
        print(f"상위 토큰: {', '.join(query_top_tokens)}")
        
        # 2. 상품 토큰 중요도 분석  
        print("\n📊 상품 토큰 중요도:")
        value_df = analyzer.visualize_token_importance(value, top_k=8)
        value_top_tokens = value_df['token'].head(3).tolist()
        print(f"상위 토큰: {', '.join(value_top_tokens)}")
        
        # 3. 쿼리 어텐션 히트맵
        print("\n📊 쿼리 어텐션 히트맵:")
        analyzer.visualize_attention_heatmap(query, layer_idx=-1, head_idx=0)
        
        results.append({
            'index': idx,
            'query': query,
            'value': value,
            'query_top_tokens': query_top_tokens,
            'value_top_tokens': value_top_tokens,
            'query_analysis': query_df,
            'value_analysis': value_df
        })
        
        print(f"✅ 샘플 {i+1} 분석 완료\n")
    
    print("🎉 모든 샘플 분석 완료!")
    return analyzer, results


# 사용 예시
if __name__ == "__main__":
    # test_df가 있다고 가정하고 사용
    # 원래 학습 파이프라인과 정확히 동일
    unique_dates = sorted(result_df['date'].unique())
    train_dates = unique_dates[:-1]  # 마지막 날짜 제외
    test_dates = [unique_dates[-1]]  # 마지막 날짜만 테스트

    train_df = result_df[result_df['date'].isin(train_dates)]
    test_df = result_df[result_df['date'].isin(test_dates)]\
    
    analyzer, results = analyze_test_samples(
        test_df=test_df, 
        model_path="./trained_insurance_model_retoken",
        sample_size=3)
    